In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-13 10:33:54 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
thetadata_start = '2021-01-01'

In [425]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
key = 8
with open(f'../input/profitable_weights_{key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{key}.csv').iloc[:, 1:]
ttrades__ = ttrades__[ttrades__.Ticker.isin(['BAC'])]
ttrades__#.Ticker.unique()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
6,19.0,506.0,756.0,30.275595,44.52,270.643702,0.470491,2021-01-05,2021-12-31,360 days,BAC


In [426]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
6,19.0,506.0,756.0,30.275595,44.52,270.643702,0.470491,2021-01-05,2021-12-31,360 days,BAC


In [427]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
pd.Series(weights).sort_values(ascending=False)

BAC    0.013145
dtype: float64

In [310]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 2
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.15
date, tick = '2023-07-05', 'AVGO'
date, tick = '2024-08-13', 'TSM'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': .50,
   'dte': 210,
   'moneyness_width': 0.5},
  #  {'direction': 'short',
  # 'rel_strike': .60,
  # 'dte': 270,
  # 'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

{'result': 'SUCCESSFUL',
 'data': {'long': ['TSM20250321C280'],
  'trade_id': '&L:TSM20250321C280',
  'close': 1.515}}

In [428]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash

{'BAC': 2}

In [439]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo

evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)

In [440]:
evb_backtest.portfolio.initial_capital

20000.0

In [441]:
w_map = {x: w  * 0.85 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 50
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .850,
   'dte': 300,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 300,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

# evb_backtest.portfolio.order_settings = {'type': 'naked',
#  'specifics': [{'direction': 'long',
#    'rel_strike': .750,
#    'dte': 210,
#    'moneyness_width': 0.5},

# ],
#  'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.executor.commission_rate
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 30
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price
evb_backtest.portfolio.roll_map
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price


{'BAC': 2}

In [442]:
for key  in max_cash:
    if max_cash[key]*100 > evb_backtest.portfolio.allocated_cash_map[key]:
        print(key, max_cash[key]*100, evb_backtest.portfolio.allocated_cash_map[key])

In [443]:
pd.options.display.max_rows = 15
pd.options.display.max_columns = 15
signals = evb_backtest.bars.signal_df
# series = signals.set_index('Date')['MSFT']
# series[series != 0]
signals

,Date,BAC
0,2021-01-05,1
1,2021-01-06,0
2,2021-01-07,0
3,2021-01-08,0
4,2021-01-11,0
...,...,...
254,2021-12-27,0
255,2021-12-28,0
256,2021-12-29,0
257,2021-12-30,0


In [444]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

1.0

## Problems:

- Current Problem: SELL Signal and Buy Signal are put right next to each other in the queue. Whereas, it is meant to be Sell Signal -> Order Event -> Fill Event -> Buy Signal -> Order Event -> Fill Event

Solution:
- Use a tuple of action ```python['CLOSE', 'OPEN']```
- Put first action into queue and return ffunctionality to backtester. Backtester then handles all corresponding sequence.
- Do the same for action two.

In [445]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Buy Details
Position: {'long': ['BAC20210820C28'], 'short': ['BAC20210820C32'], 'trade_id': '&L:BAC20210820C28&S:BAC20210820C32', 'close': 1.975}, Date: 2021-01-05, Signal: SignalEvent type:LONG, symbol=BAC, date:2021-01-05 00:00:00, Order Settings=None, signal_id:BAC20210105LONG
Max Contract Price: 2, Cash at Hand: 2.0111967577217347
Order Quantity 1 Cash at Hand 2.0111967577217347 Close 1.975
Processing event: ORDER
Buy Order Position: {'long': ['BAC20210820C28'], 'short': ['BAC20210820C32'], 'trade_id': '&L:BAC20210820C28&S:BAC20210820C32', 'close': 1.975} Price: 2.089480035051649 Quantity: 1 Datetime: 2021-01-05 00:00:00
Processing event: FILL
2021-01-05 00:00:00
BAC BUY
Event queue is empty, processed 4 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: M

In [446]:
from EventDriven.execution import exec_cache
idx = 10
buys = [x  for x in exec_cache['fill'].values() if x.direction == 'BUY']
buy1 = buys[idx]
print("Buy")
print(buy1.datetime,buy1.fill_cost/buy1.quantity, buy1.market_value)

sells = [x  for x in exec_cache['fill'].values() if x.direction == 'SELL']
sell1 = sells[idx]
print("Sell")
print(sell1.datetime,sell1.fill_cost/sell1.quantity, sell1.market_value, len(exec_cache['fill']))


Buy
2023-01-09 00:00:00 3.9226984249463994 7.819396849892799
Sell
2023-02-17 00:00:00 1.4710868610758878 4.452260583227663 153


In [447]:
pd.options.display.max_rows = 100
trades = evb_backtest.portfolio._trades.copy()#
len(trades)
# len(trades['SignalID'].unique())
trades[['Positions','Ticker','PnL', 'ReturnPct', 'EntryPrice', 'ExitPrice', 'EntryMarketValue', 'ExitMarketValue', 'EntryTime', 'ExitTime', 'EntryPrice', 'ExitPrice', 'Quantity', 'SignalID', 'Duration']]

,Positions,Ticker,PnL,ReturnPct,EntryPrice,ExitPrice,EntryMarketValue,ExitMarketValue,EntryTime,ExitTime,EntryPrice,ExitPrice,Quantity,SignalID,Duration
0,&L:BAC20210820C28&S:BAC20210820C32,BAC,157.137097,0.747389,210.248004,367.385101,208.948004,368.685101,2021-01-05,2021-07-21,210.248004,367.385101,1,BAC20210105LONG,197
1,&L:BAC20220617C40&S:BAC20220617C47,BAC,163.580132,0.773572,211.460655,375.040787,210.160655,376.340787,2021-07-21,2021-12-31,211.460655,375.040787,1,BAC20210105LONG,163


In [416]:
reasons = {
    x['reason']:0 for x in evb_backtest.portfolio.unprocessed_signals
}

for v in (evb_backtest.portfolio.unprocessed_signals):
    reasons[v['reason']] += 1
    if v['reason'] == 'UNAVAILABLE_CONTRACT':
        print(v)   
reasons

{'type': 'SIGNAL', 'symbol': 'META', 'datetime': Timestamp('2021-03-31 00:00:00'), 'signal_type': 'LONG', 'signal_id': 'META20210331LONG', 'order_settings': None, 'reason': 'UNAVAILABLE_CONTRACT'}


{'MAX_PRICE_TOO_LOW': 3,
 'TOO_ILLIQUID': 2,
 'UNAVAILABLE_CONTRACT': 1,
 'MONEYNESS_TOO_TIGHT': 2}

In [413]:
evb_backtest.portfolio.plot_portfolio()

In [414]:
evb_backtest.portfolio.aggregate()

Start                                                    2021-01-05 00:00:00
End                                                      2021-12-31 00:00:00
Duration                                                   360 days 00:00:00
Exposure Time [%]                                                      100.0
Equity Final [$]                                                    20352.11
Equity Peak [$]                                                 22475.602179
Return [%]                                                          3.020663
Buy & Hold Return [%]                                              35.353905
CAGR [%]                                                            3.063252
Volatility Ann. [%]                                                37.752185
Sharpe Ratio                                                        0.267471
Sortino Ratio                                                       0.414997
Skew                                                                0.211931

In [21]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

,,long,short,trade_id,close,quantity,market_value
datetime,symbol,,,,,,
2023-01-25,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,770.0,4,770.0
2023-01-26,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,844.0,4,844.0
2023-01-27,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,976.0,4,976.0
2023-01-30,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,688.0,4,688.0
2023-01-31,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,730.0,4,730.0
2023-02-01,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,956.0,4,956.0
2023-02-02,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1358.0,4,1358.0
2023-02-03,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1244.0,4,1244.0
2023-02-06,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1218.0,4,1218.0


In [22]:


""""
Why do these two return weird stuff after run?
    current_weighted_holdings
    current_positions

I can't reconcile the cost with the data (NVM, haha)

"""
# evb_backtest.portfolio.all_positions

'"\nWhy do these two return weird stuff after run?\n    current_weighted_holdings\n    current_positions\n\nI can\'t reconcile the cost with the data (NVM, haha)\n\n'

In [23]:
order_cache['2024-01-03']

KeyError: '2024-01-03'

## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [214]:
evb_backtest.trades.to_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options_{key}.csv')